# Object Localization Demonstration Notebook
### Import statements

In [1]:
import pyrealsense2 as rs
import cv2
import detect
import numpy as np
import time
import torch
import io

### Declaring constant variables, obtained from values in Intel and VRC documentation

In [2]:
focal_length = ((448.0-172.0) * 24.0) / 11.0
# ['Blue Goal', 'Blue Platform', 'Blue Robot', 'Neutral Goal', 'Red Goal', 'Red Platform', 'Red Robot', 'Ring']
width_dict = {"7":3.5, "3":12.5, "0":12.5, "4":12.5, "2":5.5, "5":5.5, "1":53.0, "6":53.0}

### Starting image pipeline from D435 camera

In [3]:
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, 640, 480, rs.format.rgb8, 30)
config.enable_stream(rs.stream.depth, 848, 480, rs.format.z16, 30)
config.enable_device_from_file('static/test-run-30-sec.bag')
pipeline.start(config)

### Object localization method

In [4]:
def obj_distance(cls, x, y, w, h, depth_frame):
    # calculating distance using trigonometric properties
    trig_distance = (width_dict[str(cls)] * focal_length)/w
    
    # extract average distance from depth map and convert to inches
    depth_distance_meters = (depth_frame.get_distance(x, y) +\
                             depth_frame.get_distance(x+2, y) +\
                             depth_frame.get_distance(x, y+2) +\
                             depth_frame.get_distance(x-2, y) +\
                             depth_frame.get_distance(x, y-2))/5.0
    depth_distance = 39.3701 * depth_distance_meters
    
    # weighting and combining localization methods
    distance = (trig_distance * .2) + (depth_distance_meters * .8) 
    
    # in the event that depthmap can't detect distance, only use trig distance
    if (depth_distance_meters == 0):
        distance = trig_distance
    
    return distance

### Localization Fusion Method

In [5]:
cam_height = 10
cam_angle = -5

In [6]:
# Demo x, y, theta
robot_coords = (30, 30, 90)

In [7]:
def obj_lf(obj_x, obj_y, obj_distance, robot_coords):
    
    # TODO
    
    return (calculated_x, calculated_y)

### Using identification and localization to process images from the pipeline

In [8]:
# Image generation
index = 0
for i in range(900):
    frames = pipeline.wait_for_frames()
    depth_frame = frames.get_depth_frame()
    color_frame = frames.get_color_frame()
    color_image = np.asanyarray(color_frame.get_data())
    color_image = cv2.cvtColor(color_image, cv2.COLOR_RGB2BGR)
    cv2.imwrite(f'bag_images/img{index}.png', color_image)
    index += 1

In [ ]:
# Inference
for current in range(190, 742):
    # extracting data from the image pipeline
#     frames = pipeline.wait_for_frames()
#     depth_frame = frames.get_depth_frame()
#     color_frame = frames.get_color_frame()
#     color_image = np.asanyarray(color_frame.get_data())

#     # IO for trained YOLOv5 model (color_img->game_objects)
#     color_image = cv2.cvtColor(color_image, cv2.COLOR_RGB2BGR)
#     cv2.imwrite('img.png', color_image)

    # Temp param removal: --view-img --save-txt --nosave
    !python detect.py --source bag_images --weights static/best_torchscript.pt --conf 0.45 --name yolo_gif --exist-ok

    # reading labels from localization output
    game_objects = []
    with open('runs/detect/yolo_obj/labels/img.txt', 'r+') as f:
        game_objects = f.readlines()
        f.truncate(0)
        f.close()

    # calculating distance for all game objects in frame
    for obj in game_objects:
        obj_array = obj.split()
        cls, x, y, w, h = int(obj_array[0]), int(round(float(obj_array[1]))), int(round(float(obj_array[2]))), float(obj_array[3]), float(obj_array[4])
            print(f'{obj.split()[0]}: {obj_distance(obj, depth_frame)}')
        distance = obj_distance(cls, x, y, w, h, depth_frame)
            location = obj_lf(x, y, obj_distance, robot_coords)

detect: weights=['static/best_torchscript.pt'], source=bag_images, imgsz=[640, 640], conf_thres=0.45, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs\detect, name=yolo_gif, exist_ok=True, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5  bfbcc68 torch 1.10.0 CPU

Loading static/best_torchscript.pt for TorchScript inference...
image 1/553 C:\Users\wellerj\Documents\github\senior-design-cv\localization\bag_images\img190.png: 640x640 8 class7s, Done. (4.660s)
image 2/553 C:\Users\wellerj\Documents\github\senior-design-cv\localization\bag_images\img191.png: 640x640 9 class7s, Done. (3.768s)
image 3/553 C:\Users\wellerj\Documents\github\senior-design-cv\localization\bag_images\img192.png: 640x640 9 class7s, Done. (3.448s)
image 4/553 C:\Users\wellerj\Documents\github\senior-design-cv\localization

detect: weights=['static/best_torchscript.pt'], source=bag_images, imgsz=[640, 640], conf_thres=0.45, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs\detect, name=yolo_gif, exist_ok=True, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5  bfbcc68 torch 1.10.0 CPU

Loading static/best_torchscript.pt for TorchScript inference...
image 1/553 C:\Users\wellerj\Documents\github\senior-design-cv\localization\bag_images\img190.png: 640x640 8 class7s, Done. (1.179s)
image 2/553 C:\Users\wellerj\Documents\github\senior-design-cv\localization\bag_images\img191.png: 640x640 9 class7s, Done. (0.924s)
image 3/553 C:\Users\wellerj\Documents\github\senior-design-cv\localization\bag_images\img192.png: 640x640 9 class7s, Done. (0.626s)
image 4/553 C:\Users\wellerj\Documents\github\senior-design-cv\localization

detect: weights=['static/best_torchscript.pt'], source=bag_images, imgsz=[640, 640], conf_thres=0.45, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs\detect, name=yolo_gif, exist_ok=True, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5  bfbcc68 torch 1.10.0 CPU

Loading static/best_torchscript.pt for TorchScript inference...
image 1/553 C:\Users\wellerj\Documents\github\senior-design-cv\localization\bag_images\img190.png: 640x640 8 class7s, Done. (0.716s)
image 2/553 C:\Users\wellerj\Documents\github\senior-design-cv\localization\bag_images\img191.png: 640x640 9 class7s, Done. (0.622s)
image 3/553 C:\Users\wellerj\Documents\github\senior-design-cv\localization\bag_images\img192.png: 640x640 9 class7s, Done. (0.443s)
image 4/553 C:\Users\wellerj\Documents\github\senior-design-cv\localization

detect: weights=['static/best_torchscript.pt'], source=bag_images, imgsz=[640, 640], conf_thres=0.45, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs\detect, name=yolo_gif, exist_ok=True, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5  bfbcc68 torch 1.10.0 CPU

Loading static/best_torchscript.pt for TorchScript inference...
image 1/553 C:\Users\wellerj\Documents\github\senior-design-cv\localization\bag_images\img190.png: 640x640 8 class7s, Done. (0.616s)
image 2/553 C:\Users\wellerj\Documents\github\senior-design-cv\localization\bag_images\img191.png: 640x640 9 class7s, Done. (0.484s)
image 3/553 C:\Users\wellerj\Documents\github\senior-design-cv\localization\bag_images\img192.png: 640x640 9 class7s, Done. (0.359s)
image 4/553 C:\Users\wellerj\Documents\github\senior-design-cv\localization

image 108/553 C:\Users\wellerj\Documents\github\senior-design-cv\localization\bag_images\img297.png: 640x640 11 class7s, Done. (0.390s)
image 109/553 C:\Users\wellerj\Documents\github\senior-design-cv\localization\bag_images\img298.png: 640x640 1 class3, 12 class7s, Done. (0.381s)
image 110/553 C:\Users\wellerj\Documents\github\senior-design-cv\localization\bag_images\img299.png: 640x640 1 class2, 7 class7s, Done. (0.370s)
image 111/553 C:\Users\wellerj\Documents\github\senior-design-cv\localization\bag_images\img300.png: 640x640 1 class2, 7 class7s, Done. (0.402s)
image 112/553 C:\Users\wellerj\Documents\github\senior-design-cv\localization\bag_images\img301.png: 640x640 5 class7s, Done. (0.383s)
image 113/553 C:\Users\wellerj\Documents\github\senior-design-cv\localization\bag_images\img302.png: 640x640 1 class2, 4 class7s, Done. (0.386s)
image 114/553 C:\Users\wellerj\Documents\github\senior-design-cv\localization\bag_images\img303.png: 640x640 3 class7s, Done. (0.385s)
image 115/553

detect: weights=['static/best_torchscript.pt'], source=bag_images, imgsz=[640, 640], conf_thres=0.45, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs\detect, name=yolo_gif, exist_ok=True, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5  bfbcc68 torch 1.10.0 CPU

Loading static/best_torchscript.pt for TorchScript inference...
image 1/553 C:\Users\wellerj\Documents\github\senior-design-cv\localization\bag_images\img190.png: 640x640 8 class7s, Done. (0.716s)
image 2/553 C:\Users\wellerj\Documents\github\senior-design-cv\localization\bag_images\img191.png: 640x640 9 class7s, Done. (0.555s)
image 3/553 C:\Users\wellerj\Documents\github\senior-design-cv\localization\bag_images\img192.png: 640x640 9 class7s, Done. (0.399s)
image 4/553 C:\Users\wellerj\Documents\github\senior-design-cv\localization

detect: weights=['static/best_torchscript.pt'], source=bag_images, imgsz=[640, 640], conf_thres=0.45, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs\detect, name=yolo_gif, exist_ok=True, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5  bfbcc68 torch 1.10.0 CPU

Loading static/best_torchscript.pt for TorchScript inference...
image 1/553 C:\Users\wellerj\Documents\github\senior-design-cv\localization\bag_images\img190.png: 640x640 8 class7s, Done. (0.568s)
image 2/553 C:\Users\wellerj\Documents\github\senior-design-cv\localization\bag_images\img191.png: 640x640 9 class7s, Done. (0.440s)
image 3/553 C:\Users\wellerj\Documents\github\senior-design-cv\localization\bag_images\img192.png: 640x640 9 class7s, Done. (0.347s)
image 4/553 C:\Users\wellerj\Documents\github\senior-design-cv\localization

detect: weights=['static/best_torchscript.pt'], source=bag_images, imgsz=[640, 640], conf_thres=0.45, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs\detect, name=yolo_gif, exist_ok=True, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5  bfbcc68 torch 1.10.0 CPU

Loading static/best_torchscript.pt for TorchScript inference...
image 1/553 C:\Users\wellerj\Documents\github\senior-design-cv\localization\bag_images\img190.png: 640x640 8 class7s, Done. (0.552s)
image 2/553 C:\Users\wellerj\Documents\github\senior-design-cv\localization\bag_images\img191.png: 640x640 9 class7s, Done. (0.433s)
image 3/553 C:\Users\wellerj\Documents\github\senior-design-cv\localization\bag_images\img192.png: 640x640 9 class7s, Done. (0.330s)
image 4/553 C:\Users\wellerj\Documents\github\senior-design-cv\localization

^C
^C
^C
^C
^C


detect: weights=['static/best_torchscript.pt'], source=bag_images, imgsz=[640, 640], conf_thres=0.45, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs\detect, name=yolo_gif, exist_ok=True, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5  bfbcc68 torch 1.10.0 CPU

Loading static/best_torchscript.pt for TorchScript inference...
image 1/553 C:\Users\wellerj\Documents\github\senior-design-cv\localization\bag_images\img190.png: 640x640 8 class7s, Done. (0.537s)
image 2/553 C:\Users\wellerj\Documents\github\senior-design-cv\localization\bag_images\img191.png: 640x640 9 class7s, Done. (0.476s)
image 3/553 C:\Users\wellerj\Documents\github\senior-design-cv\localization\bag_images\img192.png: 640x640 9 class7s, Done. (0.357s)
image 4/553 C:\Users\wellerj\Documents\github\senior-design-cv\localization